In [1]:
# 导入库
import numpy as np                                    
import pandas as pd   
import matplotlib.pyplot as plt  
import seaborn as sns

from SALib.analyze import sobol                        
from SALib.plotting.bar import plot as barplot         
from SALib.sample import saltelli                    

from sklearn.gaussian_process import GaussianProcessRegressor    
from sklearn.preprocessing import MinMaxScaler                   

In [2]:
# 读取表格
excel_file_path = "C:/Users/lenovo/Desktop/拉丁超立方采样/test.xlsx"           # 定义excel路径变量

data = pd.read_excel(excel_file_path)                        # 读取excel变量，'data'为读取excel后生成的DataFrame

col_name = list(data.columns.values)                         # 构建excel列名称列表，'col_name'为excel的列名

# print('data:'); print(data)                                # 显示data
# print('col_name:'); print(col_name)                        # 显示col_name
# print('data_shape:'); print(data.shape)                    # 显示data的规模，前行数，后列数
# print('data_rows:'); print(data.shape[0])                  # 显示data的行数
# print('data_columns:'); print(data.shape[1])               # 显示data的列数

In [ ]:
# 预操作
value = data.values                                          # 定义value变量, 为data的值
rows = data.shape[0]                                         # 获取表格行数rows
num = data.shape[1]-3                                        # 获取表格变量数num
a = data.shape[0] % (2 * num + 2)                            # 定义a为：行数取模（2×（列数-3）+2）
b = int((data.shape[0] - a) / ( 2 * num + 2))                # 定义b为：整型(行数-a)/(2×（列数-3）+2)，b为后续采样数

# print('value:'); print(value)                              # 显示value
# print('rows:'); print(rows)                                # 显示value
# print('num:'); print(num)                                  # 显示value
# print('a:'); print(a)                                      # 显示a
# print('b:'); print(b)                                      # 显示b

In [4]:
# 设置训练集
x = np.array(value[0:rows - a, 0:num])                       # 定义x,为：创建数组（value中的值，行数：0~行数-a, 列数：0~列数-3（即除最后3列的列））
x_train = x                                                  # 定义训练集x_trian = x

y = np.array(value[0:rows - a, num:27])                      # 定义y,为：创建数组（value中的值，行数：0~行数-a，列数：（即最后3列））
y_train = y                                                  # 定义训练集y_trian = y

# print('x_train:'); print(x_train)                            # 显示x_train
# print('x_train_shape:'); print(x_train.shape)                # 显示x_train.shape
# print('y_train:'); print(y_train)                            # 显示y_train
# print('y_train_shape:'); print(y_train.shape)                # 显示y_train.shape

# y_train = y_train.reshape(-1, 1)                           # 将y_trian转化为1列
# print('y_train:'); print(y_train)                          # 显示y_train
# print('y_train_shape:'); print(y_train.shape)              # 显示y_train.shape

In [5]:
# 回归准备
scaler = MinMaxScaler()                                     # 定义scaler函数，为‘数据归一化’操作
x_train = scaler.fit_transform(x_train)                     # 对x_train进行数据归一化操作

# print('x_train_scaler:'); print(x_train)                    # 显示归一化后的x_train

In [6]:
# 回归
reg = GaussianProcessRegressor()                         # 定义reg函数，为‘高斯过程回归’操作
reg_1 = reg.fit(x_train, y_train)                        # 拟合

# print(reg_1.score(x_train, y_train))                     # 显示拟合分数

In [7]:
# 敏感性分析问题描述
num_vars = num                                              # 定义变量数目num_vars：列数-3
names = col_name[:num]                                      # 定义变量名names：list1表中序号0~（列数-3）
bounds = [[0, 1] for i in range(num)]                       # 定义边界条件bounds：均为[0,1]

# print('num_vars:'); print(num_vars)                         # 显示x_train
# print('names:'); print(names)                               # 显示x_train.shape
# print('bounds:'); print(bounds)                             # 显示bounds

problem = {                                                 # 描述问题
    'num_vars': num_vars,                                   # 变量数目：列数-3
    'names': names,                                         # 变量名：list1表中序号0~（列数-3）
    'bounds': bounds   }                                    # 边界条件，均为[0, 1]       

In [ ]:
# 采样
param_values = saltelli.sample(problem, b)               # 定义param_values函数，为saltelli采样，对象为problem, 采样数为b

# print(param_values)                                    
# print(param_values.shape)

In [11]:
# 数据结构处理
param_values = np.array(param_values).reshape(-1, 1)         # 原param_values取值转化为1列
# print(param_values)
# print(param_values.shape)

param_values = scaler.fit_transform(param_values)            # 对param_values进行数据归一化操作
# print(param_values)
# print(param_values.shape)

param_values = param_values.reshape(-1, num)                  # 原param_values取值转化为num列
# print(param_values)
# print(param_values.shape)

In [12]:
# 预测
R = reg.predict(param_values)                             # 回归预测
# print(R)
# print(R.shape)

R1 = R.reshape(-1)
# print(R1)
# print(R1.shape)

In [ ]:
# 评估模型
Si = sobol.analyze(problem, R1, print_to_console = True)     # 评估模型

total_Si, first_Si, second_Si = Si.to_df()                # 将总阶效应、一阶效应、二阶效应分别输出为DataFrame
# print(total_Si)                                         # 显示总阶效应
# print(first_Si)                                         # 显示一阶效应
# print(second_Si)                                        # 显示二阶效应

In [ ]:
# 显示结果 
# print("-------------------------------------- Sobol Result --------------------------------------")
# for (key, value) in Si.items():
#     print(str(key)+':')
#     print(value)

# 输出结果
# excel_file = pd.ExcelWriter('Sobol_indices.xlsx')       
# # 定义文件变量，输入文件名
# total_Si.to_excel(excel_file, sheet_name = 'total_Si')                     # 将总阶效应输入Sheet1
# first_Si.to_excel(excel_file, sheet_name = 'first_Si')                     # 将一阶效应输入Sheet2
# second_Si.to_excel(excel_file, sheet_name = 'second_Si')                   # 将二阶效应输入Shett3
# excel_file.save()                                                          # 保存表格

# 输出图像
st = total_Si[['ST']].round(5) 
st_1 = st.copy()                                                      
st_1.sort_values('ST', inplace = True, ascending = False)
st_2 = st_1.T

plt.figure(figsize = (18, 8), dpi = 300)
ax = sns.barplot(data = st_2, orient = 'h', palette = 'Blues_r')    
plt.xlabel('Sensitivity', fontsize = 15)
plt.ylabel('Design Variable', fontsize = 15)
ax.bar_label(ax.containers[0])

# barplot(first_Si)                                         # 输出一阶效应图像
# plot.show()

# barplot(second_Si)                                        # 输出二阶效应图像
# plot.show()